<a href="https://colab.research.google.com/github/hosseinmiripy/rezume/blob/main/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# نصب کتابخانه‌ها در Google Colab
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install pmdarima
!pip install xgboost


  Using cached ccxt-4.4.50-py2.py3-none-any.whl.metadata (117 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.1 MB/s eta 0:00:00


In [5]:
import ccxt
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import threading
import time
import os
from sklearn.linear_model import LinearRegression
from pmdarima import auto_arima
from multiprocessing import Pool
import threading
import time
from multiprocessing import Pool
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random

In [ ]:

# --- تنظیمات تلگرام ---
TELEGRAM_API_TOKEN = '7989027484:AAHtz4q396jSU2XmGMDOW27vjkmsDvN0i8w'
CHANNEL_ID = '@hosseinbtb'

# --- ارسال پیام به تلگرام ---
# def send_message_to_telegram(message):
#     url = f'https://api.telegram.org/bot{TELEGRAM_API_TOKEN}/sendMessage'
#     payload = {'chat_id': CHANNEL_ID, 'text': message, 'parse_mode': 'Markdown'}
#     response = requests.post(url, data=payload)
#     return response
def send_message_to_telegram(message):
    # چک کردن وجود کلمه "تایم فریم" در متن
    if 'تایم‌فریم' in message:
        url = f'https://api.telegram.org/bot{TELEGRAM_API_TOKEN}/sendMessage'
        payload = {'chat_id': CHANNEL_ID, 'text': message, 'parse_mode': 'Markdown'}
        response = requests.post(url, data=payload)
        return response
    else:
        print("کلمه 'تایم فریم' در متن موجود نیست. هیچ پیامی ارسال نمی‌شود.")
        return None
# --- ارسال نمودار به تلگرام ---
def send_chart_to_telegram(chart_path):
    url = f'https://api.telegram.org/bot{TELEGRAM_API_TOKEN}/sendPhoto'
    payload = {'chat_id': CHANNEL_ID}
    files = {'photo': open(chart_path, 'rb')}
    response = requests.post(url, data=payload, files=files)
    return response

def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_market_trend(df):
    df['MA_50'] = df['close'].rolling(window=50).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    rsi = calculate_rsi(df)

    if df['MA_50'].iloc[-1] > df['MA_200'].iloc[-1] and rsi.iloc[-1] < 70:
        return "صعودی"
    elif df['MA_50'].iloc[-1] < df['MA_200'].iloc[-1] and rsi.iloc[-1] > 30:
        return "نزولی"
    return "خنثی"


def calculate_correlation(main_df, alt_df):
    """
    محاسبه همبستگی بین بیت‌کوین و ارز دیگر
    """
    main_df = main_df['close'].pct_change()
    alt_df = alt_df['close'].pct_change()
    correlation = main_df.corr(alt_df)
    return correlation

def integrate_market_trend(symbol, df, market_trend):
    signal = ""
    if market_trend == "صعودی":
        signal += f"📈 بازار کلی صعودی است. احتمال سودآوری خرید بیشتر است.\n"
    elif market_trend == "نزولی":
        signal += f"📉 بازار کلی نزولی است. احتمال سودآوری فروش بیشتر است.\n"
    else:
        signal += f"🔍 بازار خنثی است. احتیاط کنید.\n"
    return signal

def fetch_market_sentiment():
    try:
        response = requests.get("https://api.alternative.me/fng/", timeout=10)
        data = response.json()
        sentiment = data['data'][0]['value_classification']
        return f"🌟 احساسات بازار: {sentiment}"
    except requests.exceptions.RequestException:
        return "⚠️ اطلاعات احساسات بازار در دسترس نیست."

def generate_signal_with_trend(df, fibonacci_levels, symbol, market_trend):
    """
    تولید سیگنال با در نظر گرفتن روند بازار
    """
    price = df['close'].iloc[-1]
    signal = f"🔍 تحلیل {symbol}:\n"
    signal += f"📈 روند کلی بازار: {market_trend}\n"

    for level, value in fibonacci_levels.items():
        if abs(price - value) < 0.02 * price:
            if market_trend == "صعودی":
                signal += f"📈 سیگنال خرید: قیمت نزدیک به سطح فیبوناچی {level}\n"
            elif market_trend == "نزولی":
                signal += f"📉 سیگنال فروش: قیمت نزدیک به سطح فیبوناچی {level}\n"
    return signal

# --- استخراج داده‌ها ---
def fetch_data(symbol, timeframe, limit=1000, max_retries=5, delay=2):
    """
    استخراج داده‌ها با تلاش مجدد در صورت بروز خطای زمان پاسخگویی
    """
    exchange = ccxt.kucoin()
    retries = 0
    while retries < max_retries:
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            return df
        except ccxt.RequestTimeout as e:
            print(f"⚠️ خطای زمان پاسخگویی برای {symbol} در تایم‌فریم {timeframe}. تلاش مجدد... ({retries + 1}/{max_retries})")
            retries += 1
            time.sleep(delay + random.uniform(0, 1))  # تاخیر با زمان تصادفی برای کاهش فشار روی API
        except Exception as e:
            print(f"🚨 خطای غیرمنتظره برای {symbol}: {e}")
            break

    print(f"❌ تلاش‌ها برای دریافت داده‌های {symbol} در تایم‌فریم {timeframe} به پایان رسید.")
    return None


# --- محاسبه سطوح فیبوناچی ---
def calculate_fibonacci_levels(df):
    high_price = df['high'].max()
    low_price = df['low'].min()
    diff = high_price - low_price

    # بررسی اینکه نوسان بازار کافی باشد
    if diff < (high_price * 0.01):  # اگر تغییر قیمت کمتر از 1% باشد
        return {}

    levels = {
        '0%': high_price,
        '23.6%': high_price - 0.236 * diff,
        '38.2%': high_price - 0.382 * diff,
        '50%': high_price - 0.5 * diff,
        '61.8%': high_price - 0.618 * diff,
        '100%': low_price
    }
    return levels

# --- شناسایی پرایس اکشن موفق ---
def identify_price_action_pattern(df):
    # شناسایی Pin Bar صعودی و نزولی
    def is_pin_bar(df, i):
        body = abs(df['open'].iloc[i] - df['close'].iloc[i])
        upper_wick = df['high'].iloc[i] - max(df['open'].iloc[i], df['close'].iloc[i])
        lower_wick = min(df['open'].iloc[i], df['close'].iloc[i]) - df['low'].iloc[i]

        # شرایط برای Pin Bar
        if body < upper_wick and lower_wick > 2 * body:
            return "Pin Bar صعودی" if df['close'].iloc[i] > df['open'].iloc[i] else "Pin Bar نزولی"
        return None

    # شناسایی Engulfing
    def is_engulfing(df, i):
        if df['close'].iloc[i] > df['open'].iloc[i] and df['open'].iloc[i - 1] > df['close'].iloc[i - 1] and \
                df['close'].iloc[i] > df['open'].iloc[i - 1] and df['open'].iloc[i] < df['close'].iloc[i - 1]:
            return "Engulfing صعودی"
        elif df['close'].iloc[i] < df['open'].iloc[i] and df['open'].iloc[i - 1] < df['close'].iloc[i - 1] and \
                df['close'].iloc[i] < df['open'].iloc[i - 1] and df['open'].iloc[i] > df['close'].iloc[i - 1]:
            return "Engulfing نزولی"
        return None

    # شناسایی Doji
    def is_doji(df, i):
        body = abs(df['open'].iloc[i] - df['close'].iloc[i])
        upper_wick = df['high'].iloc[i] - max(df['open'].iloc[i], df['close'].iloc[i])
        lower_wick = min(df['open'].iloc[i], df['close'].iloc[i]) - df['low'].iloc[i]

        # شرایط برای Doji
        if body <= (upper_wick + lower_wick) * 0.1:
            return "Doji"
        return None

    for i in range(1, len(df)):
        pin_bar = is_pin_bar(df, i)
        engulfing = is_engulfing(df, i)
        doji = is_doji(df, i)

        # اگر الگوی مشخصی پیدا شد، آن را باز می‌گردانیم
        if pin_bar:
            return pin_bar
        elif engulfing:
            return engulfing
        elif doji:
            return doji

    return None

# --- تحلیل حجم معاملات ---
def analyze_volume(df):
    avg_volume = df['volume'].mean()
    last_volume = df['volume'].iloc[-1]

    # تحلیل حجم معاملات
    if last_volume > avg_volume * 1.5:  # حجم بالا
        return "حجم معاملات بالا"
    elif last_volume < avg_volume * 0.5:  # حجم پایین
        return "حجم معاملات پایین"
    return "حجم معاملات عادی"

# اضافه کردن ماژول‌های جدید
def fetch_order_book(symbol):
    exchange = ccxt.kucoin()
    order_book = exchange.fetch_order_book(symbol)
    return order_book

def analyze_order_book(order_book):
    large_bid_orders = [order for order in order_book['bids'] if order[1] > 100]
    large_ask_orders = [order for order in order_book['asks'] if order[1] > 100]

    if len(large_bid_orders) > len(large_ask_orders):
        return "خرید نهنگ شناسایی شد (سفارش‌های بزرگ خرید)"
    elif len(large_ask_orders) > len(large_bid_orders):
        return "فروش نهنگ شناسایی شد (سفارش‌های بزرگ فروش)"
    return "سفارش‌های عادی"

def calculate_vwap(df):
    cumulative_price_volume = (df['close'] * df['volume']).cumsum()
    cumulative_volume = df['volume'].cumsum()
    vwap = cumulative_price_volume / cumulative_volume
    return vwap.iloc[-1]

def calculate_obv(df):
    obv = [0]
    for i in range(1, len(df)):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            obv.append(obv[-1] + df['volume'].iloc[i])
        elif df['close'].iloc[i] < df['close'].iloc[i-1]:
            obv.append(obv[-1] - df['volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return obv[-1]

def analyze_on_chain_data():
    try:
        response = requests.get("https://api.some-onchain-data-provider.com/large-transactions", timeout=10)
        response.raise_for_status()
        data = response.json()
        large_transactions = [tx for tx in data if tx['value'] > 1000000]
        if large_transactions:
            return f"🚨 تراکنش‌های بزرگ شناسایی شد: {len(large_transactions)} تراکنش"
        return "تراکنش‌های عادی"
    except requests.exceptions.RequestException:
        return "⚠️ داده‌های زنجیره‌ای در دسترس نیست."


def analyze_order_flow(df):
    avg_price_change = df['close'] - df['open']
    order_flow = "خنثی"
    if avg_price_change.iloc[-1] > 0:
        order_flow = "جریان خرید"
    elif avg_price_change.iloc[-1] < 0:
        order_flow = "جریان فروش"
    return order_flow

# --- به‌روزرسانی تحلیل و پیش‌بینی ---
def analyze_and_predict(symbol, timeframe, account_balance, use_market_trend=False):
    df = fetch_data(symbol, timeframe)
    if df is None:
        print(f"❌ داده‌ای برای {symbol} دریافت نشد.")
        return


    fibonacci_levels = calculate_fibonacci_levels(df)
    order_book = fetch_order_book(symbol)  # استخراج دفتر سفارش
    whale_activity = detect_whale_activity(df)

    # انتخاب نوع سیگنال بر اساس آرگومان
    if use_market_trend:
        market_trend = calculate_market_trend(df)
        signal = generate_signal_with_trend(df, fibonacci_levels, symbol, market_trend)
    else:
        signal = generate_signal(df, fibonacci_levels, symbol, order_book)

    # ارسال تحلیل به تلگرام
    if whale_activity:
        send_message_to_telegram(f"🔍 {whale_activity}\nنماد: {symbol} | تایم‌فریم: {timeframe}")
    if signal and signal != "هیچ سیگنالی شناسایی نشد.":
        plot_and_send_chart(df, fibonacci_levels, signal, timeframe, symbol)

# کدهای دیگر بدون تغییر باقی می‌مانند
def generate_signal(df, fibonacci_levels, symbol, order_book, timeframe):
    signal = f"🔍 تحلیل {symbol}:\n"

    price = df['close'].iloc[-1]
    pattern = identify_price_action_pattern(df)  # شناسایی الگوهای پرایس اکشن
    volume_analysis = analyze_volume(df)  # تحلیل حجم معاملات
    order_book_analysis = analyze_order_book(order_book)  # تحلیل دفتر سفارش
    order_flow = analyze_order_flow(df)  # تحلیل جریان سفارش
    vwap = calculate_vwap(df)  # محاسبه VWAP
    obv = calculate_obv(df)  # محاسبه OBV
    rsi = calculate_rsi(df)  # محاسبه RSI
    on_chain_analysis = analyze_on_chain_data()  # تحلیل داده‌های زنجیره‌ای

    signal = f"🔍 تحلیل {symbol}:\n"
    signal += f"- دفتر سفارش: {order_book_analysis}\n"
    signal += f"- جریان سفارش: {order_flow}\n"
    signal += f"- VWAP: {vwap:.2f}\n"
    signal += f"- OBV: {obv} (شاخص جریان حجم. OBV منفی به معنای فشار فروش است.)\n"
    signal += f"- RSI: {rsi.iloc[-1]:.2f}\n"
    signal += f"- داده‌های زنجیره‌ای: {on_chain_analysis}\n"

    entry_price_buy, stop_loss_buy, take_profit_buy = calculate_entry_exit_ai(df, fibonacci_levels, "buy")
    entry_price_sell, stop_loss_sell, take_profit_sell = calculate_entry_exit_ai(df, fibonacci_levels, "sell")

    # افزودن اطلاعات مربوط به نقاط ورود و خروج به سیگنال
    signal += f"📈 سیگنال buy:\n"
    signal += f"- نقطه ورود: {entry_price_buy} دلار\n"
    signal += f"- حد ضرر: {stop_loss_buy} دلار\n"
    signal += f"- حد سود: {take_profit_buy} دلار\n"

    signal += f"📉 سیگنال sell:\n"
    signal += f"- نقطه ورود: {entry_price_sell} دلار\n"
    signal += f"- حد ضرر: {stop_loss_sell} دلار\n"
    signal += f"- حد سود: {take_profit_sell} دلار\n"

    # بررسی نزدیکی قیمت به سطوح فیبوناچی و تحلیل VWAP و OBV
    for level, value in fibonacci_levels.items():
        if abs(price - value) < 0.02 * price:  # بررسی نزدیکی قیمت به سطوح فیبوناچی
            if price < vwap and rsi.iloc[-1] < 33 and obv > 0 and order_flow == "جریان خرید":
                signal += f"📈 سیگنال خرید معتبر: ترکیب RSI، VWAP، OBV، و جریان سفارش تأیید شد.\n"
            elif price > vwap and rsi.iloc[-1] > 67 and obv < 0 and order_flow == "جریان فروش":
                signal += f"📉 سیگنال فروش معتبر: ترکیب RSI، VWAP، OBV، و جریان سفارش تأیید شد.\n"

    # تحلیل OBV برای تأیید سیگنال‌ها
    if obv > 0 and "خرید" in order_book_analysis:
        signal += "📈 حجم مثبت OBV شناسایی شد. خرید ممکن است مناسب باشد.\n"
    elif obv < 0 and "فروش" in order_book_analysis:
        signal += "📉 حجم منفی OBV شناسایی شد. فروش ممکن است مناسب باشد.\n"

    if pattern == "Pin Bar صعودی" and rsi.iloc[-1] < 33:
        signal += "📈 الگوی صعودی (Pin Bar) شناسایی شد. ممکن است خرید مناسب باشد.\n"
    elif pattern == "Pin Bar نزولی" and rsi.iloc[-1] > 67:
        signal += "📉 الگوی نزولی (Pin Bar) شناسایی شد. ممکن است فروش مناسب باشد.\n"
    elif pattern == "Engulfing صعودی" and rsi.iloc[-1] < 33:
        signal += "📈 الگوی صعودی Engulfing شناسایی شد. ممکن است خرید مناسب باشد.\n"
    elif pattern == "Engulfing نزولی" and rsi.iloc[-1] > 67:
        signal += "📉 الگوی نزولی Engulfing شناسایی شد. ممکن است فروش مناسب باشد.\n"
    elif pattern == "Doji" and abs(rsi.iloc[-1] - 50) < 10:
        signal += "⚖️ الگوی Doji شناسایی شد. بازار ممکن است در وضعیت بی‌طرف باشد.\n"


    # تحلیل حجم معاملات برای تقویت یا رد سیگنال
    if "حجم معاملات بالا" in volume_analysis:
        signal += "⚠️ هشدار: حجم معاملات بالا. احتمال تغییر ناگهانی در روند وجود دارد.\n"

    # تحلیل وضعیت RSI در تایم‌فریم‌های 5 دقیقه و 15 دقیقه برای تایم‌فریم‌های 1 ساعته، 4 ساعته و 1 روزه
    if timeframe in ['1h', '4h']:
        df_5m = fetch_data(symbol, '5m')
        df_15m = fetch_data(symbol, '15m')

        if df_5m is not None and df_15m is not None:
            rsi_5m = calculate_rsi(df_5m).iloc[-1]
            rsi_15m = calculate_rsi(df_15m).iloc[-1]

            signal += "\nوضعیت RSI در تایم‌فریم‌های کوچکتر:\n"
            signal += f"- RSI (5 دقیقه): {rsi_5m:.2f}\n"
            signal += f"- RSI (15 دقیقه): {rsi_15m:.2f}\n"

            if price < vwap and rsi.iloc[-1] < 33 and rsi_5m < 33 and rsi_15m < 33:
                signal += "📈 تأیید بیشتر: RSI در تایم‌فریم‌های کوچکتر نیز نشان‌دهنده خرید است.\n"
            elif price > vwap and rsi.iloc[-1] > 67 and rsi_5m > 67 and rsi_15m > 67:
                signal += "📉 تأیید بیشتر: RSI در تایم‌فریم‌های کوچکتر نیز نشان‌دهنده فروش است.\n"
            else:
                signal += "⚠️ هشدار: RSI در تایم‌فریم‌های کوچکتر وضعیت خنثی دارد.\n"
        else:
            signal += "⚠️ داده‌های تایم‌فریم‌های 5 دقیقه و 15 دقیقه در دسترس نیست.\n"

    # ارسال سیگنال نهایی به تلگرام اگر معتبر باشد
    if "سیگنال خرید" in signal or "سیگنال فروش" in signal:
        send_message_to_telegram(signal)
    else:
        signal = "هیچ سیگنالی شناسایی نشد."

    return signal


def calculate_risk_management_ai(entry_price, stop_loss, take_profit, account_balance, risk_percent=2):
    volatility = abs(entry_price - stop_loss)  # محاسبه نوسانات
    risk_amount = account_balance * (risk_percent / 100)
    position_size = risk_amount / volatility
    return round(position_size, 2)

# تابع برای محاسبه ATR
def calculate_atr(df, window=14):
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift(1)).abs()
    low_close = (df['low'] - df['close'].shift(1)).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr.iloc[-1]

# تابع برای شناسایی نقاط حمایت و مقاومت
def calculate_support_resistance(df):
    high = df['high'].max()
    low = df['low'].min()
    return low, high

# تابع برای محاسبه سطوح فیبوناچی
def calculate_fibonacci_levels(df):
    high_price = df['high'].max()
    low_price = df['low'].min()
    fibonacci_levels = {
        "level_0": low_price,
        "level_23_6": low_price + 0.236 * (high_price - low_price),
        "level_38_2": low_price + 0.382 * (high_price - low_price),
        "level_50": low_price + 0.5 * (high_price - low_price),
        "level_61_8": low_price + 0.618 * (high_price - low_price),
        "level_100": high_price
    }
    return fibonacci_levels

# تابع برای پیش‌بینی قیمت با مدل XGBoost
def xgboost_predict(df, look_back=5):
    try:
        if len(df) < look_back:
            raise ValueError("⚠️ داده‌های کافی برای پیش‌بینی موجود نیست.")

        # استخراج قیمت‌ها
        close_prices = df['close'].values
        high_prices = df['high'].values
        low_prices = df['low'].values

        # محاسبه سطوح فیبوناچی
        fibonacci_levels = calculate_fibonacci_levels(df)

        # محاسبه ATR
        atr = calculate_atr(df)

        # ایجاد داده‌های ویژگی (X) و برچسب‌ها (y)
        X = []
        y = []

        for i in range(look_back, len(close_prices)):
            # استفاده از قیمت‌های گذشته به‌عنوان ویژگی
            features = close_prices[i-look_back:i]
            features = np.append(features, [atr])  # اضافه کردن ATR به ویژگی‌ها
            for level in fibonacci_levels.values():
                features = np.append(features, level)  # اضافه کردن سطوح فیبوناچی
            X.append(features)
            y.append(close_prices[i])

        X = np.array(X)
        y = np.array(y)

        # تقسیم داده‌ها به مجموعه‌های آموزشی و آزمایشی
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # ساخت مدل XGBoost
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5)
        model.fit(X_train, y_train)

        # پیش‌بینی قیمت آینده
        predicted_price = model.predict(X_test[-1:].reshape(1, -1))

        return predicted_price[0]

    except Exception as e:
        print(f"خطا در پیش‌بینی قیمت با XGBoost: {e}")
        return None

# تابع برای محاسبه ATR
def calculate_atr(df, window=14):
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift(1)).abs()
    low_close = (df['low'] - df['close'].shift(1)).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr.iloc[-1]

# تابع برای شناسایی نقاط حمایت و مقاومت
def calculate_support_resistance(df):
    high = df['high'].max()
    low = df['low'].min()
    return low, high

def calculate_entry_exit_ai(df, fibonacci_levels, signal_type, risk_reward_ratio=2, look_back=5):
    # بررسی وجود سطوح فیبوناچی
    if not fibonacci_levels:
        raise ValueError("⚠️ سطوح فیبوناچی موجود نیست.")

    # قیمت فعلی
    current_price = df['close'].iloc[-1]

    # پیش‌بینی قیمت بعدی با مدل XGBoost
    predicted_price = xgboost_predict(df, look_back=look_back)
    if predicted_price is None:
        raise ValueError("⚠️ پیش‌بینی قیمت ناموفق بود.")

    # محاسبه ATR برای تعیین حد ضرر و حد سود
    atr = calculate_atr(df)

    # شناسایی نقاط حمایت و مقاومت
    support, resistance = calculate_support_resistance(df)

    # محاسبه حد ضرر و حد سود با استفاده از ATR و سطوح حمایت/مقاومت
    if signal_type == "buy":
        # حد ضرر پایین‌تر از قیمت فعلی و نزدیک به سطح حمایت
        stop_loss = max(support, current_price - atr)  # می‌توان ATR را به عنوان فاصله برای حد ضرر استفاده کرد
        # حد سود با نسبت ریسک/بازده
        take_profit = current_price + (current_price - stop_loss) * risk_reward_ratio
    elif signal_type == "sell":
        # حد ضرر بالاتر از قیمت فعلی و نزدیک به سطح مقاومت
        stop_loss = min(resistance, current_price + atr)  # می‌توان ATR را به عنوان فاصله برای حد ضرر استفاده کرد
        # حد سود با نسبت ریسک/بازده
        take_profit = current_price - (stop_loss - current_price) * risk_reward_ratio
    else:
        raise ValueError("⚠️ نوع سیگنال باید 'buy' یا 'sell' باشد.")

    # نقطه ورود بهینه (برای ساده‌سازی می‌توان از قیمت فعلی به‌عنوان نقطه ورود استفاده کرد)
    entry_price = current_price

    # بررسی نزدیکی قیمت فعلی به سطوح فیبوناچی برای تایید
    for level, fib_price in fibonacci_levels.items():
        if abs(current_price - fib_price) < 0.02 * current_price:
            # اگر قیمت نزدیک به سطح فیبوناچی باشد
            if signal_type == "buy" and current_price < predicted_price:
                entry_price = fib_price  # تغییر نقطه ورود به نزدیک‌ترین سطح فیبوناچی
            elif signal_type == "sell" and current_price > predicted_price:
                entry_price = fib_price  # تغییر نقطه ورود به نزدیک‌ترین سطح فیبوناچی

    return round(entry_price, 2), round(stop_loss, 2), round(take_profit, 2)


# --- مدیریت سرمایه ---
def calculate_risk_management(entry_price, stop_loss, take_profit, account_balance, risk_percent=2):
    risk_amount = account_balance * (risk_percent / 100)
    position_size = risk_amount / abs(entry_price - stop_loss)
    return round(position_size, 2)

# --- رسم نمودار و ارسال سیگنال ---
def plot_and_send_chart(df, fibonacci_levels, signal, timeframe, symbol, entry_price=None, stop_loss=None, take_profit=None, rsi=None, vwap=None):
    plt.figure(figsize=(12, 6))

    # رسم نمودار قیمت
    plt.plot(df['close'], label='قیمت بسته شدن', color='blue', linewidth=1.5)

    # رسم سطوح فیبوناچی
    for level, value in fibonacci_levels.items():
        plt.axhline(value, label=f'سطح فیبوناچی {level}', linestyle='--', alpha=0.6)

    # اضافه کردن VWAP به نمودار
    if vwap:
        plt.axhline(vwap, label='VWAP', color='orange', linestyle='-.')

    # اضافه کردن نقاط ورود، حد ضرر و حد سود
    if entry_price:
        plt.scatter(len(df) - 1, entry_price, color='green', label='نقطه ورود', zorder=5)
    if stop_loss:
        plt.scatter(len(df) - 1, stop_loss, color='red', label='حد ضرر', zorder=5)
    if take_profit:
        plt.scatter(len(df) - 1, take_profit, color='gold', label='حد سود', zorder=5)

    # رسم نمودار RSI (در صورت موجود بودن)
    if rsi is not None:
        plt.twinx()  # محور Y دوم برای RSI
        plt.plot(rsi, label='RSI', color='purple', linestyle='--', alpha=0.7)
        plt.axhline(30, color='green', linestyle='--', alpha=0.5, label='RSI اشباع فروش')
        plt.axhline(70, color='red', linestyle='--', alpha=0.5, label='RSI اشباع خرید')

    # تنظیمات نمودار
    plt.title(f'پیش‌بینی قیمت {symbol} ({timeframe})')
    plt.xlabel('زمان')
    plt.ylabel('قیمت')
    plt.legend(loc='upper left')

    # ذخیره نمودار و ارسال به تلگرام
    sanitized_symbol = symbol.replace('/', '_')
    chart_path = f'chart_{sanitized_symbol}_{timeframe}.png'

    plt.savefig(chart_path)
    plt.close()

    # ارسال نمودار و پیام به تلگرام
    send_chart_to_telegram(chart_path)
    os.remove(chart_path)

    if signal != "هیچ سیگنالی شناسایی نشد.":
        send_message_to_telegram(f"📊 نتیجه تحلیل {symbol} در تایم‌فریم {timeframe}:\n{signal}")


def detect_whale_activity(df):
    """
    شناسایی فعالیت نهنگ‌ها با بررسی حجم و نوسانات قیمت
    """
    # محاسبه میانگین و انحراف معیار حجم
    avg_volume = df['volume'].mean()
    std_volume = df['volume'].std()

    # بررسی حجم معاملات غیرمعمول (بیش از 2 برابر انحراف معیار از میانگین)
    high_volume = df['volume'].iloc[-1] > avg_volume + 2 * std_volume

    # بررسی تغییرات قیمت ناگهانی
    price_change = abs(df['close'].iloc[-1] - df['close'].iloc[-2])
    avg_price_change = abs(df['close'] - df['close'].shift(1)).mean()
    std_price_change = abs(df['close'] - df['close'].shift(1)).std()

    # تغییرات قیمت ناگهانی (بیش از 2 برابر انحراف معیار از میانگین تغییرات)
    sudden_price_move = price_change > avg_price_change + 2 * std_price_change

    # بررسی آیا حجم بالا و تغییر قیمت ناگهانی همزمان رخ داده‌اند
    if high_volume and sudden_price_move:
        return "🚨 فعالیت نهنگ شناسایی شد: حجم بالا و تغییرات ناگهانی قیمت."
    elif high_volume:
        return "⚠️ حجم معاملات غیرمعمول شناسایی شد: ممکن است فعالیت نهنگ باشد."
    elif sudden_price_move:
        return "⚠️ تغییرات ناگهانی قیمت شناسایی شد: ممکن است فعالیت نهنگ باشد."
    return None  # بازگشت مقدار None در صورت عدم شناسایی

def process_analysis(args):
    symbol, timeframe, account_balance = args
    print(f"تحلیل {symbol} در تایم‌فریم {timeframe}")

    # داده‌ها را استخراج کنید
    df = fetch_data(symbol, timeframe)
    fibonacci_levels = calculate_fibonacci_levels(df)
    order_book = fetch_order_book(symbol)  # استخراج دفتر سفارش

    # تولید سیگنال
    signal = generate_signal(df, fibonacci_levels, symbol, order_book, timeframe)  # اضافه کردن آرگومان order_book

    # whale_activity = detect_whale_activity(df)  # شناسایی فعالیت نهنگ‌ها

    # ارسال پیام فقط در صورت شناسایی فعالیت نهنگ
    # if whale_activity and "فعالیت نهنگ" in whale_activity:
    #     end_message_to_telegram(f"🔍 {whale_activity}\nنماد: {symbol} | تایم‌فریم: {timeframe}")

    if signal != "هیچ سیگنالی شناسایی نشد.":
        plot_and_send_chart(df, fibonacci_levels, signal, timeframe, symbol)


# --- رابط وب (داشبورد مدیریت) ---
# تنظیمات اولیه
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'SUI/USDT', 'DOGE/USDT', 'SHIB/USDT', 'TON/USDT', 'PEPE/USDT',  'SOL/USDT', 'LTC/USDT', 'MAJOR/USDT', 'ATOM/USDT', 'OP/USDT', 'ENA/USDT', 'MEME/USDT', 'XRP/USDT']
timeframes = ['5m', '15m', '1h', '4h']
account_balance = 100  # موجودی حساب به دلار

# مدیریت تحلیل‌ها
def schedule_analysis(symbols, timeframes, account_balance):
    while True:
        tasks = [(symbol, timeframe, account_balance) for symbol in symbols for timeframe in timeframes]
        with Pool(processes=4) as pool:
            pool.map(process_analysis, tasks)
        print("منتظر تحلیل بعدی...")
        time.sleep(5)

# اجرای سرور
if __name__ == '__main__':
    schedule_analysis(symbols, timeframes, account_balance)

تحلیل ADA/USDT در تایم‌فریم 5mتحلیل SUI/USDT در تایم‌فریم 5m
تحلیل BTC/USDT در تایم‌فریم 5m
تحلیل ETH/USDT در تایم‌فریم 5m

تحلیل ETH/USDT در تایم‌فریم 15m
تحلیل ADA/USDT در تایم‌فریم 15m
تحلیل BTC/USDT در تایم‌فریم 15m
تحلیل SUI/USDT در تایم‌فریم 15m
تحلیل ADA/USDT در تایم‌فریم 1h
کلمه 'تایم فریم' در متن موجود نیست. هیچ پیامی ارسال نمی‌شود.
تحلیل ETH/USDT در تایم‌فریم 1h
تحلیل BTC/USDT در تایم‌فریم 1h
تحلیل SUI/USDT در تایم‌فریم 1h
تحلیل ADA/USDT در تایم‌فریم 4h
